In [13]:
import os
import glob
import imageio.v3 as iio
import numpy as np

# --------------------------------------------------
# Load mask (RGB -> 2D)
# --------------------------------------------------
mask = iio.imread('cnrpark_mask_original_img_1000_750_bw.png')

# Convert RGB mask to 2D if needed
if mask.ndim == 3:
    # Use first channel (all channels are identical in BW masks)
    mask = mask[:, :, 0]

# Normalize mask values
mask = mask.copy()
mask[mask > 1] = 128   # foreground / masked area

assert mask.ndim == 2, "Mask must be 2D after conversion"


# --------------------------------------------------
# Mask application
# --------------------------------------------------
def mask_and_crop(img, mask):
    """
    Apply a binary mask to an RGB image.
    Masked pixels are painted gray (128).
    """
    assert img.ndim == 3 and img.shape[2] == 3
    assert mask.ndim == 2
    assert img.shape[:2] == mask.shape

    out = img.copy()

    out[:, :, 0][mask == 128] = 128
    out[:, :, 1][mask == 128] = 128
    out[:, :, 2][mask == 128] = 128

    # Optional cropping (disabled by default)
    # out = out[210:560, 50:-40, :]

    return out


# --------------------------------------------------
# Process images
# --------------------------------------------------
image_dir = '../../assets/demo_images'
output_prefix = 'cropped_'

file_list = glob.glob(os.path.join(image_dir, '*.jpg'))

for path in file_list:
    try:
        img = iio.imread(path)

        # Safety: ensure RGB
        if img.ndim == 2:
            img = np.stack([img] * 3, axis=-1)

        name = os.path.splitext(os.path.basename(path))[0]
        final = mask_and_crop(img, mask)

        out_path = os.path.join(
            image_dir,
            f'{output_prefix}{name}.jpg'
        )

        iio.imwrite(out_path, final)

    except Exception as e:
        print(f'failed on {path}:\n{e}')
